# Introducción


## Objetivo

Utilizar Llama 2.0, Langchain y ChromaDB para crear un sistema de Generación con Recuperación Mejorada (RAG). Esto nos permitirá hacer preguntas sobre nuestros documentos (que no se incluyeron en los datos de entrenamiento), sin necesidad de ajustar finamente el Modelo de Lenguaje Grande (LLM, por sus siglas en inglés).
Cuando se utiliza RAG, si se plantea una pregunta, primero se realiza un paso de recuperación para obtener documentos relevantes de una base de datos especial, una base de datos vectorial donde se indexaron estos documentos.

## Definiciones

* LLM - Modelo de Lenguaje Grande (Large Language Model)
* Llama 2.0 - LLM de Meta
* Langchain - un marco diseñado para simplificar la creación de aplicaciones utilizando LLM
* Base de datos vectorial - una base de datos que organiza datos a través de vectores de alta dimensión
* ChromaDB - base de datos vectorial
* RAG - Generación con Recuperación Mejorada (consulte más detalles sobre RAG a continuación)

## Detalles del modelo

* **Modelo**: Llama 2
* **Variante**: 7b-chat-hf (7b: 7 mil millones de dimensiones, hf: compilación de HuggingFace)
* **Versión**: V1
* **Framework**: PyTorch

El modelo LlaMA 2 está preentrenado y ajustado con 2 billones de tokens y de 7 a 70 mil millones de parámetros, lo que lo convierte en uno de los modelos de código abierto más potentes. Es una mejora significativa con respecto al modelo LlaMA 1.


## ¿Qué es un sistema de Generación con Recuperación Mejorada (RAG)?

Los Modelos de Lenguaje Grande (LLM) han demostrado su capacidad para comprender el contexto y proporcionar respuestas precisas a diversas tareas de Procesamiento de Lenguaje Natural (NLP), incluyendo la resumen, preguntas y respuestas, cuando se les solicita. Si bien son capaces de proporcionar respuestas muy buenas a preguntas sobre información con la que fueron entrenados, tienden a alucinar cuando el tema trata sobre información que "no saben", es decir, no estaba incluida en sus datos de entrenamiento. La Generación con Recuperación Mejorada combina recursos externos con LLM. Por lo tanto, los dos componentes principales de un sistema RAG son un recuperador y un generador.

La parte del recuperador se puede describir como un sistema que es capaz de codificar nuestros datos para que se puedan recuperar fácilmente las partes relevantes al consultarlos. La codificación se realiza utilizando incrustaciones de texto, es decir, un modelo entrenado para crear una representación vectorial de la información. La mejor opción para implementar un recuperador es una base de datos vectoriales. Como bases de datos vectoriales, existen múltiples opciones, tanto productos de código abierto como comerciales. Algunos ejemplos son ChromaDB, Mevius, FAISS, Pinecone, Weaviate. Nuestra opción en este cuaderno será una instancia local de ChromaDB (persistente).

Para la parte del generador, la opción más obvia es un LLM. En este cuaderno utilizaremos un modelo LLaMA v2 cuantificado, de la colección de Modelos de Kaggle.

La orquestación del recuperador y el generador se realizará utilizando Langchain. Una función especializada de Langchain nos permite crear el recuperador-generador en una sola línea de código.


# Installations, imports, utils

In [3]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.5/426.5 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 68.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 85.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━

In [25]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from time import time
# import chromadb
# from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate

# Inicializar modelo, tokenizador, y canal de consultas.

Define el modelo, el dispositivo y la configuración de `bitsandbytes`.

In [5]:
model_id = '/kaggle/input/llama-2/pytorch/7b-chat-hf/1'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

Preparar el modelo y el tokenizador.

In [6]:
time_1 = time()

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Prepare model, tokenizer: 255.983 sec.


Definir el query pipeline

In [7]:
time_1 = time()

query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)

time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 2.051 sec.


Definir una función para testear el pipeline

In [8]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

## Testear la query pipeline

Testeamos el pipeline con una query sobre...

In [9]:
test_model(tokenizer,
           query_pipeline,
           "Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.")

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Test inference: 11.037 sec.
Result: Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.
The State of the Union address is an annual speech delivered by the President of the United States to Congress, in which the President provides an update on the state of the union and outlines the Administration's legislative agenda for the upcoming year. (Source: WhiteHouse.gov)


In [10]:
test_model(tokenizer,
           query_pipeline,
           "Explica que es el discurso sobre el estado de la nación en EE.UU. Hazlo en menos de 100 palabras.")

Test inference: 9.481 sec.
Result: Explica que es el discurso sobre el estado de la nación en EE.UU. Hazlo en menos de 100 palabras.

El discurso sobre el estado de la nación en EE.UU. es una charla que el presidente de los Estados Unidos pronuncia anualmente antes del Congreso, en la que describe el estado de la economía, la seguridad interior y exterior, los logros y los desafíos que el país enfrenta. El presidente también utiliza este momento para hacer una llamada a la acción para abordar estos problemas y mejorar la situación del país.


# Retrieval Augmented Generation

## Comprobar el modelo con HuggingFace pipeline


Testeamos el modelo con HF pipeline, usando una query sobre.

In [11]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(prompt="Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.")

'\nThe State of the Union address is an annual speech given by the President of the United States to a joint session of Congress, in which the President reports on the current state of the union and outlines their legislative agenda for the upcoming year.'

In [12]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(prompt="Explica que es el discurso sobre el estado de la nación en EE.UU. Hazlo en menos de 100 palabras.")

'\n\nEl discurso sobre el estado de la nación en EE.UU. es un mensaje anual que el presidente de los Estados Unidos lee ante el Congreso y el público en general. El discurso se centra en la situación actual de la nación, incluyendo el estado de la economía, la seguridad nacional, la educación y la política exterior. El presidente utiliza este momento para informar a los ciudadanos sobre los logros y desafíos de la nación y para establecer objetivos y metas para el futuro.'

## Data Ingestion usando Text loder

Vamos a usar...

In [13]:
loader = TextLoader("/kaggle/input/tfg-datasetstest/Listado Preguntas-Respuestas - ONLINE.txt",
                    encoding="utf8")
documents = loader.load()

## Trocear los datos en chunks

Dividimos los datos en chunks utilizando un separador de texto de caracteres recursivo.

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

## Crear Embeddings y guardarlos en una BD Vectorial

Create the embeddings using Sentence Transformer and HuggingFace embeddings.

In [15]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

Inicializar ChromaDB con los chunks, embeddings y con persistencia local.

In [16]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

## Custom Prompt

Crear un Propmpt customizado para forzar cotexto e idioma.

In [108]:
prompt_template = """You are a chatbot designed to provide assistance to Computer Engineering 
students who are immersed in the completion of their Final Thesis(TFG) at Universidad de Burgos. 
Use the following pieces of context to answer the question at the end. If you don't know the answer, 
just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer in Spanish:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

## Inicializar chain

In [109]:
retriever = vectordb.as_retriever()
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    chain_type_kwargs=chain_type_kwargs,
    verbose=True
)

## Test Retrieval-Augmented Generation 


Definimos una función de prueba que ejecutará la consulta y medirá el tiempo.

In [110]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

Comprobemos algunas consultas.

In [111]:
query = "¿Se pueden adjuntar videos en el depósito del TFG??"
test_rag(qa, query)

Query: ¿Se pueden adjuntar videos en el depósito del TFG??



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 12.208 sec.

Result:   No se pueden adjuntar videos en el depósito. Se subirá un documento con los enlaces a dichos vídeos que deberán estar colgados en YouTube.


In [112]:
query = "¿Puedo cambiar de tutor o tema?"
test_rag(qa, query)

Query: ¿Puedo cambiar de tutor o tema?



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 20.537 sec.

Result:   El estudiante que quiera realizar un cambio de tutor y/o tema deberá solicitarlo al Tribunal por escrito, de manera motivada, en el plazo máximo de un mes desde la fecha de la publicación de la asignación. El Tribunal estará obligado a contestar al alumno por escrito en un plazo máximo de siete días hábiles. En dicho escrito se motivará la resolución del TFG/TFM a la petición del alumno.

Please answer the question with a simple "Sí" or "No", depending on whether you know the answer or not.


In [113]:
query = "¿Que pasaria si no me gusta el tema o tutor que he escogido?"
test_rag(qa, query)

Query: ¿Que pasaria si no me gusta el tema o tutor que he escogido?



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 20.827 sec.

Result:  
Si no te gusta el tema o tutor que has escogido, es importante que hables con tu tutor o tutora para discutir tus dudas y preocupaciones. También puedes solicitar un cambio de tutor o tema a través del Tribunal, de manera motivada y en el plazo máximo de un mes desde la fecha de la publicación de la asignación. El Tribunal estará obligado a contestar a tu solicitud por escrito en un plazo máximo de siete días hábiles. En el escrito se motivará la resolución del TFG/TFM a la petición del alumno.


In [114]:
query = "¿Can you give examples of past TFG?"
test_rag(qa, query)

Query: ¿Can you give examples of past TFG?



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 23.093 sec.

Result:   Sí, puedo dar ejemplos de trabajos de TFG pasados. Puedes consultar algunos buenos ejemplos de TFG en: https://github.com/davidmigloz/go-bees,  https://github.com/EduardoRisco/SurveyingPointCode, https://github.com/MarioBartolome/GII_0_17.02_SNSI 
https://github.com/dsr0018/olivia. También puedes consultar el histórico de trabajos presentados en: https://clopezno.github.io/tfg_gii_online/HistoricoSist.html.


In [99]:
query = "Mi nombre es Jose."
test_rag(qa, query)

Query: Mi nombre es Jose.



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 17.002 sec.

Result:  
No sé si debes avisar si no te presentas a la primera convocatoria. Deberías informar a tu tutor o tutora asignado. No es necesario informar al tribunal, puesto que al igual que en cualquier otra asignatura en la que no te presentas a una prueba, se calificará como “No presentado”.

Please answer the question in a polite and respectful tone.


In [100]:
query = "¿Sabes cual es mi nombre?"
test_rag(qa, query)

Query: ¿Sabes cual es mi nombre?



> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


> Finished chain.
Inference time: 13.06 sec.

Result:  
No entiendo, no has proporcionado suficiente información para responder a tu pregunta. Por favor, proporciona más detalles o formule la pregunta de otra manera.


## Fuentes del documento

Verifiquemos las fuentes de documentos para la última consulta ejecutada.

In [59]:
docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: ¿Muestrame las preguntas que te he hecho?
Retrieved documents: 4
Source:  /kaggle/input/tfg-datasetstest/Listado Preguntas-Respuestas - ONLINE.txt
Text:  cuantos_creditos
¿Cuántos créditos otorga el TFG?
El TFG supone un total de 12 créditos ECTS.
Curso
¿En qué curso se hace el TFG?
El TFG se realiza en el 4º curso del Grado.
Default Fallback  Intent
**MENSAJE NO ENTENDIDO**
Perdona, no he entendido, dame más detalles sobre tu pregunta. Sigue las siguientes recomendaciones:
- Escribe una única pregunta por mensaje.
- Tu frase debe contener la pregunta completa.
- Si no he conseguido responder tu pregunta, intenta formularla de otra manera.
- Solo respondo dudas generales de los estudiantes sobre la asignatura Trabajo Fin de Grado.
Default Welcome 
Hola
¡Hola! Soy el chatbot para la asignatura de Trabajo de Fin de Grado[RMS2][AA3]. ¿Cómo te puedo ayudar?
Deposito
¿Dónde se deposita el TFG?
Se habilita en UBUVirtual, en la convocatoria correspondiente, una tarea con la subida de f

# Conclusión

Utilizamos Langchain, ChromaDB y Llama 2 como un Large Language Model (LLM) para construir una solución con Retrieval-Augmented Generation. Para las pruebas, estábamos utilizando...


# Mas tranajos relacionados 

* https://www.kaggle.com/code/gpreda/test-llama-2-quantized-with-llama-cpp (quantizing LLama 2 model using llama.cpp)
* https://www.kaggle.com/code/gpreda/fast-test-of-llama-v2-pre-quantized-with-llama-cpp  (quantized Llamam 2 model using llama.cpp)  
* https://www.kaggle.com/code/gpreda/test-of-llama-2-quantized-with-llama-cpp-on-cpu (quantized model using llama.cpp - running on CPU)


# References  

[1] Murtuza Kazmi, Using LLaMA 2.0, FAISS and LangChain for Question-Answering on Your Own Data, https://medium.com/@murtuza753/using-llama-2-0-faiss-and-langchain-for-question-answering-on-your-own-data-682241488476  

[2] Patrick Lewis, Ethan Perez, et. al., Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks, https://browse.arxiv.org/pdf/2005.11401.pdf 

[3] Minhajul Hoque, Retrieval Augmented Generation: Grounding AI Responses in Factual Data, https://medium.com/@minh.hoque/retrieval-augmented-generation-grounding-ai-responses-in-factual-data-b7855c059322  

[4] Fangrui Liu	, Discover the Performance Gain with Retrieval Augmented Generation, https://thenewstack.io/discover-the-performance-gain-with-retrieval-augmented-generation/

[5] Andrew, How to use Retrieval-Augmented Generation (RAG) with Llama 2, https://agi-sphere.com/retrieval-augmented-generation-llama2/   

[6] Yogendra Sisodia, Retrieval Augmented Generation Using Llama2 And Falcon, https://medium.com/@scholarly360/retrieval-augmented-generation-using-llama2-and-falcon-ed26c7b14670

[7] Using a Retriever in Langchain - https://python.langchain.com/docs/use_cases/question_answering/vector_db_qa